In [ ]:
%%time
#
# Criado vetores TF-IDF a partir de corpus criado de Perimeter.log
import glob
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer

root_dir = '/home/ricardo/pucrs/TCC/logs-corpus/'
tipo_log = 'Perimeter.log'

corpus = glob.glob(os.path.join(root_dir, tipo_log, '*'))
print(f'Qtd documentos no corpus de {tipo_log}: {len(corpus)}')
vectorizer = TfidfVectorizer(input='filename', stop_words='english', smooth_idf = True, ngram_range=(1,1),
                             token_pattern=r'(?u)\b\w*[a-zA-Z]\w+\b')
X = vectorizer.fit_transform(corpus)
print(X.shape)

In [ ]:
%%time
from tqdm import tqdm
import os
import glob
from sklearn import svm

corpus = glob.glob(os.path.join(root_dir, tipo_log, '*'))
print(f'Quantidade de arquivos: {len(corpus)}')

gamma    = .2
nu       = .1
oneclass = svm.OneClassSVM(kernel='rbf', gamma=gamma, nu=nu)
oneclass.fit(X)


In [ ]:
# Para validação do modelo mais adequado, 
# recupera 10000 registros de logs aleatorios que nao sejam do tipo Perimeter.log
from tqdm import tqdm
from pymongo import MongoClient
import shutil

def geraMassaTesteAleatoria():
    # Replace the placeholder with your connection string.
    uri = "mongodb://localhost:27017/"
    client = MongoClient(uri)
    
    root_dir = '/home/ricardo/pucrs/TCC/logs-corpus/validacao-aleatoria/'
    
    if os.path.exists(root_dir):
       shutil.rmtree(root_dir)
    
    os.mkdir(root_dir)  
    
    try:
        agg_db = client["logdb"]
    
        logs_coll = agg_db["logs"]
        
        pipeline = []    
    
        pipeline.append({
            '$match': {'arq_nome': {'$ne': 'Perimeter.log'}}
        })
        
        pipeline.append({
            '$sample': {'size': 10000}
        })
    
        aggregation_result = logs_coll.aggregate(pipeline)
    
        for document in tqdm(aggregation_result):
            file = open(os.path.join(root_dir,document['arq_nome']+'-'+str(document['_id'])), 'w', encoding='utf-8') 
            try:
                file.write(document['texto'])
            finally:
                file.close()
    
    finally:
        client.close()

In [ ]:
%%time
import glob
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer

root_dir = '/home/ricardo/pucrs/TCC/logs-corpus/validacao-aleatoria'

for x in range(0,10):
    print(f'-------- Inicio interacao {x+1}')
    geraMassaTesteAleatoria()
    corpus_validacao = glob.glob(os.path.join(root_dir, '*'))
    print(f'Qtd documentos para validacao aleatoria: {len(corpus_validacao)}')
    #
    predicoes = []
    for log in tqdm(corpus_validacao):
        X_validar = vectorizer.transform([log])
        predicao = oneclass.predict(X_validar)
        predicoes.append(predicao[0])
        
    corretas_anomalias = predicoes.count(-1)
    acuracia         = (corretas_anomalias/len(predicoes))
    print(f'Corretas anomalias: {corretas_anomalias}')
    print(f'Acuracia: {acuracia}')
    #
    print(f'gamma={gamma} nu={nu} corretas_anomalias={corretas_anomalias} acuracia={acuracia}')
    print(f'-------- Fim interacao {x+1}')

In [ ]:
len(predicoes)